In [1]:
import os
from pydub import AudioSegment
import whisper

c:\Users\Jason\anaconda3\envs\General\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
def chunk_audio(file_path, chunk_length_ms=60000):
    audio = AudioSegment.from_file(file_path, format="mp3")
    length_audio = len(audio)
    chunks = []
    for i in range(0, length_audio, chunk_length_ms):
        chunks.append(audio[i:i+chunk_length_ms])
    return chunks

def format_timestamp(seconds):
    hours, remainder = divmod(int(seconds), 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"[{hours:02d}:{minutes:02d}:{seconds:02d}]"

def transcribe_chunk(chunk, model, start_time):
    chunk.export("temp_chunk.mp3", format="mp3")
    result = model.transcribe("temp_chunk.mp3")
    os.remove("temp_chunk.mp3")
    
    transcriptions = []
    for segment in result['segments']:
        timestamp = format_timestamp(start_time + segment['start'])
        transcriptions.append(f"{timestamp} {segment['text']}")
    
    return '\n'.join(transcriptions), start_time + result['duration']

def transcribe_audio(file_path, chunk_length_ms=60000):
    model = whisper.load_model("base")  # or any other model size
    audio_chunks = chunk_audio(file_path, chunk_length_ms)
    directory = os.path.dirname(file_path)
    output_file = os.path.join(directory, "transcription_output.txt")
    
    start_time = 0
    with open(output_file, 'w') as f:
        for i, chunk in enumerate(audio_chunks):
            try:
                print(f"Transcribing chunk {i+1}/{len(audio_chunks)}...")
                transcription, start_time = transcribe_chunk(chunk, model, start_time)
                f.write(transcription + '\n')
            except Exception as e:
                print(f"Error transcribing chunk {i+1}: {e}")


In [5]:
path_to_mp3 = "/home/jason/Desktop/Study/Transcripts/Test Lectures/Cpp_Course.mp3"
transcribe_audio(path_to_mp3)

Transcribing chunk 1/143...


/home/jason/anaconda3/envs/llama/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1702400366987/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,


Transcribing chunk 2/143...
Transcribing chunk 3/143...
Transcribing chunk 4/143...
Transcribing chunk 5/143...
Transcribing chunk 6/143...
Transcribing chunk 7/143...
Transcribing chunk 8/143...
Transcribing chunk 9/143...
Transcribing chunk 10/143...
Transcribing chunk 11/143...
Transcribing chunk 12/143...
Transcribing chunk 13/143...
Transcribing chunk 14/143...
Transcribing chunk 15/143...
Transcribing chunk 16/143...
Transcribing chunk 17/143...
Transcribing chunk 18/143...
Transcribing chunk 19/143...
Transcribing chunk 20/143...
Transcribing chunk 21/143...
Transcribing chunk 22/143...
Transcribing chunk 23/143...
Transcribing chunk 24/143...
Transcribing chunk 25/143...
Transcribing chunk 26/143...
Transcribing chunk 27/143...
Transcribing chunk 28/143...
Transcribing chunk 29/143...
Transcribing chunk 30/143...
Transcribing chunk 31/143...
Transcribing chunk 32/143...
Transcribing chunk 33/143...
Transcribing chunk 34/143...
Transcribing chunk 35/143...
Transcribing chunk 36/